In [130]:
import pandas as pd
import numpy as np
import re
import networkx as nx
import helpers as hpr

In [3]:
df = pd.DataFrame([])
data_path = "Changes/"
changes_file_names = hpr.list_file(data_path)
for i in range(len(changes_file_names)):
    # df_per_file = pd.read_csv("%s/%s" % (data_path, f))
    df_per_file = pd.read_csv("%schanges_data_%d.csv" % (data_path, i))
    df = pd.concat((df, df_per_file))

In [206]:
df = df.reset_index(drop=True).sort_values(by="updated", ascending=False)

In [418]:
df = pd.read_csv("./experiments/os_datasets.csv")

In [419]:
len(df)

703048

In [420]:
(df[["number"]].value_counts() == 2).sum() # No emty id

85

In [421]:
df = df.drop_duplicates(subset=["number"])

In [422]:
(df[["number"]].value_counts() == 2).sum()

0

In [159]:
def retrieve_depends_on(x):
    result = re.split(r"\n|\r", x)

    final_result = []
    for l in result:
        if re.search(r"Depends-On:\s[a-zA-Z0-9]+", l):
            if l.startswith("Depends-On: http://review") | l.startswith("Depends-On: https://review"):
                final_result.append(str(re.search(r"\d+", l)[0]))
            elif re.search(r"^Depends-On:\s[^@:%._\\+~#?&//=\\s]", l):
                if l.find("http") != -1:
                    continue
                final_result.append(l[12:].split(" ")[0])
    return final_result if len(final_result) != 0 else None


def retrieve_related_bug(x):
    result = re.search(r"(Related-Bug:\s#\d+)", x)
    return result[0][14:] if result else None

In [ ]:
# 110705,"['I01c0ecc276a73fbbc06ec39bfaa1c66f47fc3a9c Trove', 'Iee7ed7b141548ef64231acad6319e769d356b97f python-troveclient']"
# 110704,['I01c0ecc276a73fbbc06ec39bfaa1c66f47fc3a9c Trove']
# 628498,"['Iae415339308a93ad49eeb7e4d7e0be2662abf0ee', 'I7762af7b824ae0c7303438d48fc35c9f24a00c9c']"
# 642605,['639096']
# 118521,['Id22d18a84a4ac104ff8af09959e8eb2ae0102e97']
# 124181,['I3a772e84a1bc6028caeabcc8e09a0942f44b8612']

# 637664,"['635634', '635621', '635648']"

In [8]:
test_df = df[df["number"].isin([
    110705, 110704, 628498, 642605, 118521, 124181, 637664
])].copy().reset_index(drop=True)


In [9]:
test_df["depends_on"] = test_df["commit_message"].map(retrieve_depends_on)
test_df["related_bug"] = test_df["commit_message"].map(retrieve_related_bug)

In [35]:
test_df.loc[test_df["number"]==110705, "commit_message"].to_numpy()
# test_df.loc[test_df["depends_on"].isnull() == False, ["number", "depends_on"]].to_csv("./experiments/test.csv", index=False)

array(['Add mgmt API for datastores\r\n\r\nImplemented mgmt API calls for create/update datastores/versions.\r\nTo access this API call "admin" role is required.\r\nRemoved datastores related commands from trove-manage.\r\nChanged response from datastore-list, now regular user can see only\r\ndatastores which has atleast one active version.\r\n\r\nChanges:\r\n - updated the way of datastores are being registered\r\n   while calling ./redstack kick-start <datastore>\r\n\r\nPartially implements: blueprint datastores-mgmt-api\r\n\r\nDepends-On: I01c0ecc276a73fbbc06ec39bfaa1c66f47fc3a9c Trove\r\nDepends-On: Iee7ed7b141548ef64231acad6319e769d356b97f python-troveclient\r\n\r\nChange-Id: I49195c2c8c08fe4673b2dc69d86ea16ee7654431\r\n'],
      dtype=object)

In [160]:
df["depends_on"] = df["commit_message"].map(retrieve_depends_on)
df["related_bug"] = df["commit_message"].map(retrieve_related_bug)

In [161]:
df = df.explode(column="depends_on").reset_index(drop=True)

In [162]:
df[["id"]].describe()

,id
count,707145
unique,702912
top,openstack%2Ftripleo-ci~master~I2c0726b2234e0b7...
freq,44


In [163]:
len(df)

707145

In [164]:
df[["depends_on"]].value_counts().to_csv("./experiments/test.csv")

In [165]:
def flatten_list(array):
    response = [item for sublist in array for item in sublist]
    return response

In [167]:
def build_depends_chain(row):
    obj = {}
    depends_on = row["depends_on"]
    obj["Target"] = row["number"]
    obj["Target_repo"] = row["project"]
    row_src = None
    if depends_on.isnumeric():
        row_src = df[df["number"] == int(depends_on)]
    else:
        row_src = df[df["change_id"] == depends_on]

    if len(row_src) != 0:
        source_numbers = flatten_list(row_src[["number"]].to_numpy())

        source_numbers = list(dict.fromkeys(source_numbers))
        obj["Source"] = source_numbers
        obj["Source_repo"] = row_src["project"].head(1).tolist()[0]

    return obj

In [60]:
df_subset_columns = ["change_id", "project", "depends_on", "number"]

df_subset_dep = df.loc[df["depends_on"].isnull() == False,
                       df_subset_columns].copy().reset_index(drop=True)

df_depends_on = df_subset_dep.apply(build_depends_chain, axis=1)

df_depends_on = pd.json_normalize(data=df_depends_on, errors="ignore")

In [63]:
df_depends_on.isnull().sum()

Target            0
Target_repo       0
Source         1183
Source_repo    1183
dtype: int64

In [65]:
df_depends_on = df_depends_on.dropna()

In [66]:
df_depends_on.isnull().sum()

Target         0
Target_repo    0
Source         0
Source_repo    0
dtype: int64

In [70]:
df_depends_on = df_depends_on.explode(column="Source").reset_index(drop=True)

In [71]:
len(df_depends_on)

36225

In [73]:
evolution_columns = ["Source", "Target", "Source_repo", "Target_repo"]

df_depends_on = df_depends_on.loc[:, evolution_columns]

In [81]:
df_depends_on[df_depends_on["Source_repo"] == df_depends_on["Target_repo"]].count()

Source         3243
Target         3243
Source_repo    3243
Target_repo    3243
dtype: int64

In [82]:
df_depends_on = df_depends_on[df_depends_on["Source_repo"] != df_depends_on["Target_repo"]]

In [83]:
df_depends_on[df_depends_on["Source_repo"] == df_depends_on["Target_repo"]].count()

Source         0
Target         0
Source_repo    0
Target_repo    0
dtype: int64

In [84]:
df_depends_on.count()

Source         32982
Target         32982
Source_repo    32982
Target_repo    32982
dtype: int64

In [86]:
df_depends_on = df_depends_on.reset_index(drop=True)

In [87]:
df_depends_on["Source"] = df_depends_on[["Source"]].astype(int)
df_depends_on["Target"] = df_depends_on[["Target"]].astype(int)

In [88]:
df_depends_on.to_csv("./Files/clean_openstack_evolution.csv", index=False)

### Building Data for graph

In [138]:
df_depends_on = pd.read_csv("./Files/clean_openstack_evolution.csv")

In [139]:
graph = nx.from_pandas_edgelist(df=df_depends_on, source="Source", target="Target", create_using=nx.DiGraph)

In [140]:
from itertools import chain, product, starmap
from functools import partial


roots = (node for node, d in graph.in_degree if d == 0)

leaves = (node for node, d in graph.out_degree if d == 0)

all_paths = partial(nx.all_simple_paths, graph)

paths = list(chain.from_iterable(starmap(all_paths, product(roots, leaves))))

In [145]:
pd.DataFrame({"Path": paths}).to_csv("./Files/co_evolution_number_depends_on.csv", index=False) 

In [146]:
os_edges = df_depends_on[["Source", "Target"]].copy()
os_nodes_1 = df_depends_on[["Source", "Source_repo"]].copy()
os_nodes_1 = os_nodes_1.rename(columns={
    "Source": "id",
    "Source_repo": "label"
})
os_nodes_2 = df_depends_on[["Target", "Target_repo"]].copy()
os_nodes_2 = os_nodes_2.rename(columns={
    "Target": "id",
    "Target_repo": "label"
})

In [147]:
os_edges = os_edges.reset_index(drop=True)

In [148]:
os_nodes = pd.concat((os_nodes_1, os_nodes_2))
os_nodes = os_nodes.drop_duplicates()
os_nodes = os_nodes.reset_index(drop=True)

### Related-Bug

In [148]:
def combine_projects(row):
    combined_projects = ""
    projects = list(dict.fromkeys(row["project"]))
    for i in range(len(projects)):
        combined_projects += projects[i]
        if i + 1 < len(projects):
            combined_projects += " "
    return combined_projects

In [150]:
def retrieve_project_numbers(x):
    return x["number"].values

def remove_single_components(arr):
    result = []
    for item in arr:
        if len(dict.fromkeys(item)) > 1:
            result.append(list(item))

    return result

In [168]:
df_main_related_bug = df[df["related_bug"].isnull() == False].copy()

df_main_related_bug["related_bug"] = df_main_related_bug[["related_bug"]].astype(int).reset_index(drop=True)

df_related_bug_subset = df_main_related_bug.copy()[[
    "related_bug", "number"
]].groupby("related_bug").apply(retrieve_project_numbers).reset_index(
    drop=True)

related_bug_number_co_changes = remove_single_components(df_related_bug_subset)

### Topic

In [169]:
df_main_topic = df[df["topic"].isnull() == False].copy()

df_topic_subset = df_main_topic.copy()[[
    "topic", "number"
]].groupby("topic").apply(retrieve_project_numbers).reset_index(
    drop=True)

topic_number_co_changes = remove_single_components(df_topic_subset)

### Subject

In [170]:
df_main_subject = df[df["subject"].isnull() == False].copy()

df_subject_subset = df_main_subject.copy()[[
    "subject", "number"
]].groupby("subject").apply(retrieve_project_numbers).reset_index(
    drop=True)

subject_number_co_changes = remove_single_components(df_subject_subset)

### Change-id

In [171]:
df_main_change_id = df[df["change_id"].isnull() == False].copy()

df_change_id_subset = df_main_change_id.copy()[[
    "change_id", "number"
]].groupby("change_id").apply(retrieve_project_numbers).reset_index(
    drop=True)

change_id_number_co_changes = remove_single_components(df_change_id_subset)

### All possible paths including depend-on, related-bug, topic, subject and change-id based co-changes

In [326]:
def combine_co_changes_number(depends_on_number_co_changes, other_arrays):
    result = depends_on_number_co_changes.copy()
    
    for other_arr in other_arrays:
        for arr_item in other_arr:
            arrays_to_add = []

            for j in range(len(depends_on_number_co_changes)):
                row_dep = depends_on_number_co_changes[j]
                check_any = any(item in row_dep for item in arr_item)

                if check_any == True:
                    result[j] = list(dict.fromkeys(result[j] + arr_item))
                elif arr_item not in arrays_to_add:
                    arrays_to_add.append(arr_item)

            arrays_to_add = [item for item in arr_item]
            result.append(arrays_to_add)
            arrays_to_add = []
    return result

### Test

In [435]:
depends_on_test = [["A", "C"], ["D", "C"], ["F", "K"]]
other_arrays_test = [["B", "C"], ["A", "F"], ["L", "K"], ["Z", "N"]]

test = combine_co_changes_number(depends_on_test, [other_arrays_test])
print(test)

[['A', 'C', 'B', 'F'], ['D', 'C', 'B'], ['F', 'K', 'A', 'L'], ['B', 'C'], ['A', 'F'], ['L', 'K'], ['Z', 'N']]


In [300]:
result_number_co_changes = combine_co_changes_number(paths, [related_bug_number_co_changes])

In [301]:
result_number_co_changes = combine_co_changes_number(result_number_co_changes, [topic_number_co_changes])

In [302]:
result_number_co_changes = combine_co_changes_number(result_number_co_changes, [subject_number_co_changes])

In [303]:
result_number_co_changes = combine_co_changes_number(result_number_co_changes, [change_id_number_co_changes])

In [308]:
pd.DataFrame({"Path": paths}).to_csv("./Co-changes/Number/depends_on.csv", index=False)
pd.DataFrame({"Path": related_bug_number_co_changes}).to_csv("./Co-changes/Number/related_bug.csv", index=False)
pd.DataFrame({"Path": topic_number_co_changes}).to_csv("./Co-changes/Number/topic.csv", index=False)
pd.DataFrame({"Path": subject_number_co_changes}).to_csv("./Co-changes/Number/subject.csv", index=False)
pd.DataFrame({"Path": change_id_number_co_changes}).to_csv("./Co-changes/Number/change_id.csv", index=False)
pd.DataFrame({"Path": result_number_co_changes}).to_csv("./Co-changes/Number/all_paths.csv", index=False)

### Simple test of the 1st item of the paths

In [332]:
print(result_number_co_changes[0])
print(list(dict.fromkeys(paths[0]+result_number_co_changes[0])))

[865314, 865315, 212127, 599335, 579277, 510663, 485956, 465150, 461122, 360999, 360991, 96492]
[865314, 865315, 212127, 599335, 579277, 510663, 485956, 465150, 461122, 360999, 360991, 96492]


### Extending the csv file with single-component related changes that do not appear in other co-changes

In [423]:
all_paths_flattend = list(dict.fromkeys(flatten_list(result_number_co_changes)))

In [424]:
single_component_changes = df.loc[~df["number"].isin(all_paths_flattend), ["number", "project"]].reset_index(drop=True)

In [426]:
single_component_changes_number = single_component_changes["number"].map(lambda x: [x]).tolist()

In [452]:
extended_paths_number = result_number_co_changes + single_component_changes_number

In [433]:
print("result_number_co_changes %d"%len(result_number_co_changes))
print("single_component_changes_number %d"%len(single_component_changes_number))
print("extended_paths_number %d"%len(extended_paths_number))

result_number_co_changes 215309
single_component_changes_number 214328
extended_paths_number 429637


In [463]:
result_number_co_changes = pd.read_csv("./Co-changes/Number/all_paths.csv", converters={'Path': pd.eval})

### Replace numbers with repos in extended_paths_number 

In [448]:
def number_to_repo(data):
    df_subset = df[["number", "project"]]
    for i in range(len(data)):
        row = data[i]
        for j in range(len(row)):
            data[i][j] = df_subset.loc[df_subset["number"] == row[j], "project"].tolist()[0]
    return data

In [ ]:
extended_paths_repo = number_to_repo(extended_paths_number)
# extended_paths_number[1]

In [505]:
result = []
# test = flatten_list(result_number_co_changes.copy().values)
test = [["A", "B", "D"], ["X", "K", "L"], ["C", "D"], ["M", "N", "Q"],
        ["X", "K"]]
for i in range(len(test)):
    new_item = {}

    if len(test[i]) == 0:
        continue

    for j in range(1, len(test)):
        check_all = all(item in test[i] for item in test[j])
        if len(test[j]) == 0 or check_all == True:
            continue

        check_any = any(item in test[i] for item in test[j])
        if check_any == True:
            new_item = new_item | dict.fromkeys(test[i] + test[j])
            test[j] = []
    
    if len(new_item) == 0:
        result.append(test[i])
    else:
        result.append(list(new_item))

    test[i] = []
print(result)

[['A', 'B', 'D', 'C'], ['X', 'K', 'L'], ['M', 'N', 'Q'], ['X', 'K']]


[865314 865315 212127 599335 579277 510663 485956 465150 461122 360999
 360991 96492]
